In [9]:
! git clone https://github.com/seshuad/IMagenet

Cloning into 'IMagenet'...
Updating files:   1% (1950/120206)
Updating files:   2% (2405/120206)
Updating files:   2% (3548/120206)
Updating files:   3% (3607/120206)
Updating files:   3% (4767/120206)
Updating files:   4% (4809/120206)
Updating files:   5% (6011/120206)
Updating files:   5% (6012/120206)
Updating files:   6% (7213/120206)
Updating files:   6% (7588/120206)
Updating files:   7% (8415/120206)
Updating files:   7% (9305/120206)
Updating files:   8% (9617/120206)
Updating files:   9% (10819/120206)
Updating files:   9% (10959/120206)
Updating files:  10% (12021/120206)
Updating files:  10% (12845/120206)
Updating files:  11% (13223/120206)
Updating files:  12% (14425/120206)
Updating files:  12% (14762/120206)
Updating files:  13% (15627/120206)
Updating files:  13% (16739/120206)
Updating files:  14% (16829/120206)
Updating files:  15% (18031/120206)
Updating files:  15% (18565/120206)
Updating files:  16% (19233/120206)
Updating files:  16% (20292/120206)
Updating files

In [11]:
%pip install imageio

Note: you may need to restart the kernel to use updated packages.


In [12]:
import time
import numpy as np
import imageio

path = 'IMagenet/tiny-imagenet-200/'

def get_id_dictionary():
    id_dict = {}
    for i, line in enumerate(open(path + 'wnids.txt', 'r')):
        id_dict[line.replace('\n', '')] = i
    return id_dict

def get_class_to_id_dict():
    id_dict = get_id_dictionary()
    all_classes = {}
    result = {}
    for i, line in enumerate(open(path + 'words.txt', 'r')):
        n_id, word = line.split('\t')[:2]
        all_classes[n_id] = word
    for key, value in id_dict.items():
        result[value] = (key, all_classes[key])
    return result

def get_data(id_dict):
    print('starting loading data')
    train_data, test_data = [], []
    train_labels, test_labels = [], []
    t = time.time()
    for key, value in id_dict.items():
        train_data += [imageio.imread(path + 'train/{}/images/{}_{}.JPEG'.format(key, key, str(i))) for i in range(500)]
        train_labels_ = np.array([[0]*200]*500)
        train_labels_[:, value] = 1
        train_labels += train_labels_.tolist()

    for line in open(path + 'val/val_annotations.txt'):
        img_name, class_id = line.split('\t')[:2]
        test_data.append(imageio.imread(path + 'val/images/{}'.format(img_name)))
        test_labels_ = np.array([[0]*200])
        test_labels_[0, id_dict[class_id]] = 1
        test_labels += test_labels_.tolist()

    print('finished loading data, in {} seconds'.format(time.time() - t))
    return np.array(train_data), np.array(train_labels), np.array(test_data), np.array(test_labels)

train_data, train_labels, test_data, test_labels = get_data(get_id_dictionary())

print("train data shape: ", train_data.shape)
print("train label shape: ", train_labels.shape)
print("test data shape: ", test_data.shape)
print("test_labels.shape: ", test_labels.shape)

starting loading data


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_20680\4056828735.py:30: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  train_data += [imageio.imread(path + 'train/{}/images/{}_{}.JPEG'.format(key, key, str(i))) for i in range(500)]
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_20680\4056828735.py:37: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  test_data.append(imageio.imread(path + 'val/images/{}'.format(img_name)))


KeyboardInterrupt: 

In [ ]:
import time
import numpy as np
import imageio


path = 'IMagenet/tiny-imagenet-200/'

def get_id_dictionary():
    id_dict = {}
    for i, line in enumerate(open(path + 'wnids.txt', 'r')):
        id_dict[line.replace('\n', '')] = i
    return id_dict

def get_class_to_id_dict():
    id_dict = get_id_dictionary()
    all_classes = {}
    result = {}
    for i, line in enumerate(open(path + 'words.txt', 'r')):
        n_id, word = line.split('\t')[:2]
        all_classes[n_id] = word
    for key, value in id_dict.items():
        result[value] = (key, all_classes[key])
    return result

def get_data(id_dict):
    print('starting loading data')
    train_data, test_data = [], []
    train_labels, test_labels = [], []
    t = time.time()
    for key, value in id_dict.items():
        train_data += [imageio.imread(path + 'train/{}/images/{}_{}.JPEG'.format(key, key, str(i))) for i in range(500)]
        train_labels_ = np.array([[0]*200]*500)
        train_labels_[:, value] = 1
        train_labels += train_labels_.tolist()

    for line in open(path + 'val/val_annotations.txt'):
        img_name, class_id = line.split('\t')[:2]
        test_data.append(imageio.imread(path + 'val/images/{}'.format(img_name)))
        test_labels_ = np.array([[0]*200])
        test_labels_[0, id_dict[class_id]] = 1
        test_labels += test_labels_.tolist()

    print('finished loading data, in {} seconds'.format(time.time() - t))
    return np.array(train_data), np.array(train_labels), np.array(test_data), np.array(test_labels)



In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
class TinyImageNetDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = self.data[idx]
        label = self.labels[idx]
        image = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1) # Convert to torch tensor and change to channel first
        label = torch.tensor(label, dtype=torch.float32)
        return image, label

In [ ]:
# Load data
train_data, train_labels, test_data, test_labels = get_data(get_id_dictionary())

# Create datasets
train_dataset = TinyImageNetDataset(train_data, train_labels)
test_dataset = TinyImageNetDataset(test_data, test_labels)

# Create dataloaders
batch_size = 4
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [2]:
from datasets import load_from_disk, Dataset, DatasetDict
from typing import Union
train_local_dir:str = 'tiny_imagenet_local/train'
val_local_dir:str = 'tiny_imagenet_local/valid'
train_ds: Union[Dataset, DatasetDict] = load_from_disk(train_local_dir)
val_ds: Union[Dataset, DatasetDict] = load_from_disk(val_local_dir)
print(train_ds)
print(val_ds)

Dataset({
    features: ['image', 'label'],
    num_rows: 100000
})
Dataset({
    features: ['image', 'label'],
    num_rows: 10000
})


In [3]:
for data in train_ds:
    print(data)
    break

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=64x64 at 0x1A5D0F76960>, 'label': 0}


In [4]:
from PIL import Image
from typing import *
first_data:Dict = train_ds[0]
image:Image = first_data['image']
label:int = first_data['label']
#Check the image!
image.show()

In [5]:
#Using numpy since it cant be converted to torch tensors directly
import numpy as np
image_array:np.array = np.array(image)
print(f'Image array shape: {image_array.shape}')
print(f'Label: {label}')

Image array shape: (64, 64, 3)
Label: 0


In [8]:
def transform_batch(batch):
    # Convert each PIL image to a list of consistent shape
    image_arrays = [np.array(img).tolist() for img in batch['image']]
    return {
        'image_array': image_arrays
    }

# Apply the transformation
dataset_dict = train_ds.map(
    transform_batch,
    batched=True,
    remove_columns=['image']  # drop the 'image' column safely
)


Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

ArrowInvalid: cannot mix list and non-list, non-null values

In [18]:
val_ds.map(mapper_fn)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset({
    features: ['image', 'label'],
    num_rows: 10000
})

In [21]:
for data in train_ds:
    print(data)
    break

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=64x64 at 0x2F94E17A600>, 'label': 0}


In [14]:
from torch.utils.data import DataLoader
train_dataloader: DataLoader = DataLoader(train_ds, batch_size = 8, shuffle = True)
val_dataloader: DataLoader = DataLoader(val_ds, batch_size = 8, shuffle = True)

In [16]:
for index, batch in enumerate(train_dataloader):
    print(f'Batch no {index+ 1}')
    print(batch)

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'PIL.JpegImagePlugin.JpegImageFile'>